In [1]:
import numpy as np
import scipy.stats as st

## Задание 2d

а) Найдем приблизительный (асимптотический) 90\%-ый доверительный интервал для $\psi$, используя многопараметрический дельта-метод

In [2]:
alpha = 0.1
z_alpha = st.norm.ppf(1 - alpha / 2)

In [3]:
n1 = n2 = 200
X1 = 160
X2 = 148

In [4]:
p1 = X1 / n1
p2 = X2 / n2

print("Оценка для вероятностей: p1 = {0}, p2 = {1}".format(p1, p2))

Оценка для вероятностей: p1 = 0.8, p2 = 0.74


In [5]:
psi_mle = p1 - p2
print("Оценка для psi =", psi_mle)

Оценка для psi = 0.06000000000000005


In [6]:
se = np.sqrt(p1*(1 - p1) / n1 + p2*(1 - p2) / n2)
eps = z_alpha * se

print("Асимптотический 90% доверительный интервал по дельта-методу: ({0}, {1})".format(psi_mle - eps, psi_mle + eps))

Асимптотический 90% доверительный интервал по дельта-методу: (-0.009044678043395274, 0.12904467804339537)


б) Найдем приблизительный (асимптотический) 90\%-ый доверительный интервал для $\psi$, используя параметрический бутстреп

In [7]:
B = 10000

sample1 = np.array([1]*X1 + [0]*(n1 - X1))
sample2 = np.array([1]*X2 + [0]*(n2 - X2))

x1_boots = np.random.choice(sample1, (n1, B), replace=True)
x2_boots = np.random.choice(sample2, (n2, B), replace=True)

hat_p1_boots = x1_boots.mean(axis=0)
hat_p2_boots = x2_boots.mean(axis=0)

hat_psi_boots = hat_p1_boots - hat_p2_boots

In [8]:
se_boots = np.sqrt(np.var(hat_psi_boots))
eps = z_alpha * se_boots

print("Асимптотический 90% доверительный интервал по параметрическому бутстрепу: ({0}, {1})".format(psi_mle - eps, psi_mle + eps))

Асимптотический 90% доверительный интервал по параметрическому бутстрепу: (-0.0085253402998786, 0.12852534029987872)


## Задание 8

В десятичной записи числа π среди первых 10002 знаков после запятой цифры 0, 1, . . . , 9 встречаются соответственно 968, 1026, 1021, 974, 1014, 1046, 1021, 970, 948, 1014 раз. Можно ли при уровне значимости 0.05 считать эти цифры случайными? При каком уровне значимости эта гипотеза отвергается?

**Решение:** Будем решать задачу с помощью критерия на основе отношения правдоподобия. Цифры являются случайными, если их вероятности равны (те вероятность появления каждой цифры равна $\frac{1}{10}$). Поэтому рассмотрим следующие гипотезы:  
- $H_0$: $p_{i} = \frac{1}{10}$, $\forall i = 0, \dots, 9$
- $H_1$: $p_{i} = \frac{X_{i}}{n}$, где $X_i$ - количество появлений i цифры $\forall i = 0, \dots, 9$

In [9]:
counts = np.array([968, 1026, 1021, 974, 1014, 1046, 1021, 970, 948, 1014])
n = counts.sum()

print("counts =", counts)
print("n =", n)

counts = [ 968 1026 1021  974 1014 1046 1021  970  948 1014]
n = 10002


In [10]:
probs = counts / n

print("probs =", probs)

probs = [0.09678064 0.10257948 0.10207958 0.09738052 0.10137972 0.10457908
 0.10207958 0.0969806  0.09478104 0.10137972]


In [11]:
l = 2 * (np.sum(counts * np.log(probs) - counts * np.log(0.1)))

print("lambda =", l)

lambda = 9.398472367755858


In [12]:
alpha = 0.05
chi_square = st.chi2.ppf(1 - alpha, 9)

print("chi_square = {}".format(chi_square))

chi_square = 16.918977604620448


$\lambda$ предельно распределена по закону $\chi_{9}^{2}$ и меньше критического значения хи-квадрат при уровне значимости 0.05 c 9 степенями свободы. Следовательно, гипотеза $H_0$ о том, что цифры являются случайными в записи числа $\pi$, принимается

Чтобы отклонить гипотезу, нужно, чтобы $\lambda > \chi_{9}^{2}$ при заданном уровне значимости $\alpha$. Найдём его.

In [13]:
alpha_ = 1 - st.chi2.cdf(l, 9)

print("Гипотеза отвергается при уровне значимости {0:.3f} и выше".format(alpha_))

Гипотеза отвергается при уровне значимости 0.401 и выше


## Задание 9

Предположим, что у нас есть 10 статей, написанных автором, скрывающемся под псевдонимом. Мы подозреваем, что эти статьи на самом деле написаны некоторым известным писателем. Чтобы проверить эту гипотезу, мы подсчитали доли четырехбуквенных слов в 8-и сочинениях подозреваемого нами автора: .224 .261 .216 .239 .229 .228 .234 .216  
В 10 сочинениях, опубликованных под псевдонимом, доли четырехбуквенных слов равны .207 .204 .195 .209 .201 .206 .223 .222 .219 .200  
• Используйте критерий Вальда. Найдите p-value и 95%-ый доверительный интервал для разницы средних
значений. Какой можно сделать вывод исходя из найденных значений?  
• Используйте критерий перестановок. Каково в этом случае значение p-value. Какой можно сделать вывод?

In [14]:
alpha = 0.05

counts_1 = np.array([.224, .261, .216, .239, .229, .228, .234, .216], dtype=np.float64)
counts_2 = np.array([.207, .204, .195, .209, .201, .206, .223, .222, .219, .200], dtype=np.float64)

#### Критерий Вальда

Рассмотрим выборочное среднее обоих выборок: $\mu_1$ для автора и $\mu_2$ для автора под псевдонимом. Рассмотрим следующие гипотезы:
- $H_0$: $\theta = 0$  
- $H_1$: $\theta \neq 0$  
где $\theta = \mu_1 - \mu_2$

Будем считать, что если выборочные средние отличаются, то статьи принадлежат разным авторам.

In [15]:
mu_1 = np.mean(counts_1)
se_1 = np.std(counts_1)

mu_2 = np.mean(counts_2)
se_2 = np.std(counts_2)

theta_hat = mu_1 - mu_2
se_hat = np.sqrt(se_1 ** 2 / counts_1.size + se_2 ** 2 / counts_2.size)

print("theta_hat = {}, se = {}".format(theta_hat, se_hat))

theta_hat = 0.02227499999999996, se = 0.0056236262211317


In [16]:
W = (theta_hat - 0) / se_hat
z = st.norm.ppf(1 - alpha / 2)
eps = z * se_hat

print("W =", W)
print("z =", z)
print("95% доверительный интервал: ({}, {})".format(theta_hat - eps, theta_hat + eps))

W = 3.9609673765831
z = 1.959963984540054
95% доверительный интервал: (0.011252895144066747, 0.03329710485593318)


$p-value = 2Ф(-|W|)$

In [17]:
p_value = 2 * st.norm.cdf(-abs(W))
print("p-value = {0:.7f}".format(p_value))

p-value = 0.0000746


$|W| > z_{\frac{\alpha}{2}}$, $\theta_{0} = 0$ не принадлежит $95\%$ доверительному интервалу, $p-value$ очень мало (меньше $0.01$). Следовательно, гипотеза о том, что статьи принадлежат данному автору отклоняется. 

#### Критерий перестановок

In [18]:
from itertools import combinations
from math import factorial


all_counts = np.concatenate([counts_1, counts_2])
statistic = 0
for combination in combinations(all_counts, counts_1.size):
    mu_1 = np.mean(combination)
    mu_2 = (all_counts.sum() - mu_1 * len(combination)) / 10
    statistic += 1 if abs(mu_1 - mu_2) > theta_hat else 0

statistic *= factorial(counts_1.size) * factorial(counts_2.size)
statistic /= factorial(all_counts.size)

In [19]:
statistic

0.0007084418849124731

При заданной статистике (меньше 0.01) можно отклонить гипотезу того, что статьи принадлежат одному автору

## Задание 10

Девочка каждый будний день путешествует в метро от станции $A$ до станции $B$. Со станции $A$ составы идут в двух направлениях: до станции $B$ и до станции $C$.
Если приходит поезд до станции $C$, Девочке приходится дожидаться следующего поезда. Девочке кажется, что ей очень везёт с поездами до станции $B$ и что поезда до станции $B$ ходят чаще, чем поезда до станции $C$, поэтому на протяжении двух месяцев записывает, до какой станции идут поезда, которые она успела увидеть, спустившись на станцию (таблица trains.csv).
Необходимо проверить, ходят ли поезда до станции $B$ значимо чаще, чем до станции $C$.

Обозначим реальную частоту поездов до станции $B$ через $p$ и будем считать, что поезда приходят случайно и независимо друг от друга. Необходимо:  
a) Построить тест на основе критерия отношения правдоподобий для различения гипотез $H_0\colon p = p_0$ vs. $H_1 \colon p \neq p_0$.  
b) Пусть $p_0 = \frac12$. Сравнить (как аналитически, так и экспериментально) полученный тест с тестом Вальда для различения этих гипотез.

In [20]:
import pandas as pd

trains = pd.read_csv('trains.csv')
trains.head()

,train_to_B
0,1
1,0
2,0
3,1
4,1


#### Критерий на основе отношения правдоподобия

In [21]:
count_to_B = trains['train_to_B'].sum()
n = trains['train_to_B'].shape[0]

p_hat = count_to_B / n

print('p = {}'.format(p_hat))

p = 0.7391304347826086


In [22]:
p_0 = 0.5
l = 2 * (count_to_B * np.log(p_hat / p_0) + (n - count_to_B) * np.log((1 - p_hat) / (1 - p_0)))

print('lambda = {}'.format(l))

lambda = 10.96480740332921


In [23]:
chi_square = st.chi2.ppf(1 - 0.05, 1)
"chi_square = {}".format(chi_square)

'chi_square = 3.841458820694124'

Значение критерия больше, чем квантиль распределения хи-квадрат с одной степенью свободы, поэтому
гипотеза о том, что $p = p_0 = \frac{1}{2}$ отклоняется тестом на основе критерия отношения правдоподобия с уровнем значимости $0.05$.

#### Тест Вальда

In [24]:
se = np.sqrt(p_hat * (1 - p_hat) / n)
W = (p_hat - p_0) / se

eps = st.norm.ppf(1 - 0.05 / 2) * se
format("95% доверительный интервал: ({}, {})".format(p_hat - eps, p_hat + eps))

'95% доверительный интервал: (0.6122363361845568, 0.8660245333806605)'

In [25]:
p_value = 2 * st.norm.cdf(-abs(W))
format("p-value = {}".format(p_value))

'p-value = 0.00022116322931875268'

Гипотеза $H_{0}: p = p_0 = \frac{1}{2}$ отклонена тестом Вальда, так как $p-value < 0.01$

На практике эквивалентность тестов показана, так как они выносят одно и тоже решение: гипотеза $H_0$ отклоняется